In [21]:
# Importaciones
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random
from datetime import datetime
from pathlib import Path

In [22]:
# Eliminar "comas" por "espacio en blanco" y "punto y coma" por "coma"
# Abrir, leer y modificar el contenido del archivo
def limpiar_archivos(file, output_file):
    with open(file, "r", encoding="utf-8") as f:
        contenido = f.read()

    # Reemplazos: primero las comas por espacio, luego los puntos y coma por coma
    contenido = contenido.replace(",", " ")
    contenido = contenido.replace(";", ",")

    # Guardar el contenido modificado en un nuevo archivo (o sobrescribir el original si quieres)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(contenido)

limpiar_archivos("ruta.csv", "rechazos_ruta.csv")
limpiar_archivos("transportista.csv", "rechazos_tr.csv")

In [23]:
# ATENCION /.\: Puede que no lea el csv si hay comas entre nombres de transportistas, etc (ELIMINAR LAS COMAS PRIMERO)
ruta = pd.read_csv('rechazos_ruta.csv')
tr = pd.read_csv('rechazos_tr.csv')

In [24]:
# eliminar todas las columnas excepto las relevantes
ruta = ruta[[
    'Locación',
    'Ruta Troncal Dinámico',
    'Motivo de anulación',
    'Número de orden de carga', # main property
    'Día',
    'Venta Perdida CF',
    'Cliente',
    'Transportista',
    'Nombre Vendedor',
]]

tr = tr[[
    'Locación', 
    'Código de Transportista',
    'Transportista',
    'Fecha',
    'Orden de Carga', # main property
    'Rechazo CF',
    'Número de Clientes Programados'
]]

In [25]:
# conservar las filas que contenga las siguientes Locaciones (por si acaso lo normal es que no esten estas locaciones)
locaciones_tr = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']
tr = tr[tr['Locación'].isin(locaciones_tr)]

locaciones_ruta = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']
ruta = ruta[ruta['Locación'].isin(locaciones_ruta)]

parse_locaciones = {
    '06 AYA EL PEDREGAL': 'El Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}

In [26]:
# rellenar los valores NaN con 0
ruta = ruta.fillna(0)
tr = tr.fillna(0)

# eliminar filas cuando la cantidad sea 0
ruta = ruta[ruta["Venta Perdida CF"] != 0]
tr = tr[tr["Rechazo CF"] != 0]

print(len(ruta))
print(len(tr))

8302
3938


In [27]:
# Establecer dia como tipo fecha
ruta['Día'] = pd.to_datetime(ruta['Día'], errors='coerce', dayfirst=True)
tr['Fecha'] = pd.to_datetime(tr['Fecha'], format='%d/%m/%Y')

# Seleccionar la opcion_tiempo de análisis por tiempo
opcion_tiempo = int(input("(1=Año) (2=Mes) (3=Dia) (4=Intervalo) (5=Mayor igual que)"))
fecha = 0

if opcion_tiempo == 1:
    fecha = input("año")
    fecha_inicio = pd.to_datetime('01/01/'+str(fecha), format='%d/%m/%Y')
    fecha_fin = pd.to_datetime('31/12/'+str(fecha), format='%d/%m/%Y')
    ruta = ruta[(ruta['Día'] >= fecha_inicio) & (ruta['Día'] <= fecha_fin)]
    tr = tr[(tr['Fecha'] >= fecha_inicio) & (tr['Fecha'] <= fecha_fin)]
elif opcion_tiempo == 2:
    fecha = input("mes/año")
    month, year = fecha.split('/')
    fecha_inicio = pd.to_datetime('01/'+str(month)+'/'+str(year), format='%d/%m/%Y')
    fecha_fin = pd.to_datetime('01/'+str(int(month)+1)+'/'+str(year), format='%d/%m/%Y')
    ruta = ruta[(ruta['Día'] >= fecha_inicio) & (ruta['Día'] < fecha_fin)]
    tr = tr[(tr['Fecha'] >= fecha_inicio) & (tr['Fecha'] < fecha_fin)]
elif opcion_tiempo == 3:
    fecha = input("dia/mes/año")
    fecha_corte = pd.to_datetime(str(fecha), format='%d/%m/%Y')
    ruta = ruta[ruta['Día'] == fecha_corte]
    tr = tr[tr['Fecha'] == fecha_corte]
elif opcion_tiempo == 4:
    fecha = input("dia/mes/año dia/mes/año")
    fecha_inicio, fecha_fin = fecha.split(' ')
    fecha_inicio = pd.to_datetime(str(fecha_inicio), format='%d/%m/%Y')
    fecha_fin = pd.to_datetime(str(fecha_fin), format='%d/%m/%Y')
    ruta = ruta[(ruta['Día'] >= fecha_inicio) & (ruta['Día'] <= fecha_fin)]
    tr = tr[(tr['Fecha'] >= fecha_inicio) & (tr['Fecha'] <= fecha_fin)]
elif opcion_tiempo == 5:
    fecha = input("dia/mes/año")
    fecha_corte = pd.to_datetime(str(fecha), format='%d/%m/%Y')
    ruta = ruta[ruta['Día'] >= fecha_corte]
    tr = tr[tr['Fecha'] >= fecha_corte]

C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_9920\2987213995.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ruta['Día'] = pd.to_datetime(ruta['Día'], errors='coerce', dayfirst=True)


In [28]:
# agrupar por 'Número de orden de carga' y 'Orden de Carga'
ruta_grouped = ruta.groupby('Número de orden de carga').agg({
    'Locación': 'first',
    'Día': 'first',
    'Venta Perdida CF': 'sum',
    'Cliente': 'first',
    'Transportista': 'first',
    'Nombre Vendedor': 'first'
}).reset_index()
print("Venta perdida x Cliente y ruta diaria: ", len(ruta_grouped))

tr_grouped = tr.groupby('Orden de Carga').agg({
    'Locación': 'first',
    'Transportista': 'first',
    'Fecha': 'first',
    'Rechazo CF': 'sum',
}).reset_index()
print('Reporte de Guías de Distribución por Fecha: ', len(tr_grouped))

Venta perdida x Cliente y ruta diaria:  201
Reporte de Guías de Distribución por Fecha:  198


In [29]:
# Renombrar columnas para que coincidan
ruta_ids = ruta_grouped[['Número de orden de carga', 'Día']].copy()
ruta_ids.columns = ['Orden', 'Fecha']
ruta_ids['Fuente xlsx'] = 'Venta perdida x Cliente y ruta diaria'
ruta_ids['Asunto Correo'] = 'Venta perdida diaria por cliente y ruta - acum mes'

tr_ids = tr_grouped[['Orden de Carga', 'Fecha']].copy()
tr_ids.columns = ['Orden', 'Fecha']
tr_ids['Fuente xlsx'] = 'Reporte de Guías de Distribución por Fecha'
tr_ids['Asunto Correo'] = 'Reporte de ordenes de carga diario'

# Concatenar ambos sets
combined = pd.concat([ruta_ids, tr_ids], ignore_index=True)

# Contar ocurrencias por Orden y Fecha
counts = combined.groupby(['Orden', 'Fecha']).size().reset_index(name='Conteo')

# Filtrar los que están en solo una fuente
diffs = counts[counts['Conteo'] == 1].drop(columns='Conteo')

# Unir de vuelta para saber a qué fuente pertenecen
resultado = diffs.merge(combined, on=['Orden', 'Fecha'], how='left')

# Ordenar por Fecha y Orden
resultado = resultado.sort_values(['Fecha', 'Orden'])

# Cambiar orden de columnas
resultado = resultado[['Fecha', 'Orden', 'Asunto Correo', 'Fuente xlsx']]

In [30]:
# Guardar concatenacion de csv
resultado.to_csv("incoherenciarRechazos.csv", index=False, encoding="utf-8-sig")